# CLASSIFICATION OF LAB SAMPLES

This notebook analyzes the accuracy of the SVM classifier on the training data and on lab samples.
The training data are:
* all genes (not just regulators);
* averaged;
* do not include T1 (since only one observtion)

# TO DO
1. Analyses for Eliza:
   1. Accuracy with all 11 and the accuracy as you add in more genes (same as what you just sent but with all 11).
   1. Could you test with just the 10 mycobactin genes (Rv2377c-Rv2386c)?
   1. Could you test with just bacterioferritin (Rv3841)? Just Rv2384?
   1. Is there one gene that gives good accuracy
1. compare accuracy with ranom selection of genes
1. Classifier coefficient profile.
   1. Eliza genes - 1-10
   1. Auto selected genes: 5, 10, 20

# Preliminaries

## Imports

In [1]:
import init
from common import constants as cn
from common.trinary_data import TrinaryData, REF_TYPE_BIOREACTOR, REF_TYPE_SELF, REF_TYPE_POOLED
from common.data_provider import DataProvider
from common import trinary_data
from common_python.plots import util_plots
from common_python.classifier import classifier_ensemble
from common_python.classifier import classifier_collection
from common import transform_data

import collections
import copy
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import cross_val_score
import seaborn as sns

%matplotlib inline

## Constants

In [29]:
PROVIDER = DataProvider()
PROVIDER.do()
DATA = TrinaryData(is_regulator=False, is_dropT1=True, is_averaged=True)
DF_X = DATA.df_X.copy()
SER_Y = DATA.ser_y
NEW_DATA = copy.deepcopy(DATA)
rename_dct = {0: 0, 1: 3, 2:1, 3: 2, 4:4}
NEW_DATA.ser_y = pd.Series([rename_dct[v] for v in SER_Y.values])
NEW_DATA.ser_y.index = DATA.ser_y.index
STAGE_NAME_DCT = {0: "Transition",  1: "Stage1a", 2: "Stage1b", 3: "StageII", 4: "Resusciation"}
STAGE_NAMES = np.array(list(STAGE_NAME_DCT.values()))

In [31]:
NEW_DATA.ser_y

timepoint
T2     0
T3     0
T4     0
T5     0
T6     0
T7     0
T8     2
T9     2
T10    3
T11    2
T12    3
T13    3
T14    3
T15    3
T16    1
T17    1
T18    1
T19    2
T20    2
T21    2
T22    4
T23    4
T24    4
T25    4
dtype: int64

In [3]:
STAGE_NAME_DCT

{0: 'Transition', 1: 'Stage1a', 2: 'Stage1b', 3: 'StageII', 4: 'Resusciation'}

In [4]:
[STAGE_NAMES[i] for i in SER_Y]

['Transition',
 'Transition',
 'Transition',
 'Transition',
 'Transition',
 'Transition',
 'StageII',
 'StageII',
 'Stage1a',
 'StageII',
 'Stage1a',
 'Stage1a',
 'Stage1a',
 'Stage1a',
 'Stage1b',
 'Stage1b',
 'Stage1b',
 'StageII',
 'StageII',
 'StageII',
 'Resusciation',
 'Resusciation',
 'Resusciation',
 'Resusciation']

In [5]:
MYCOBACTIN_GENES = [
"Rv2377c",
"Rv2378c",
"Rv2379c",
"Rv2380c",
"Rv2381c",
"Rv2382c",
"Rv2383c",
"Rv2384",
"Rv2385",
"Rv2386c",
]
BACTERIOFERRITIN_GENES = [
"Rv2341", "Rv3841", 
]
GENE_DCT = {"mycobactin": MYCOBACTIN_GENES,
            "bacterioferritin 1": [BACTERIOFERRITIN_GENES[0]],
            "bacterioferritin 2": [BACTERIOFERRITIN_GENES[1]],
           }

In [6]:
ELIZA_COLUMNS = ["Rv3841",
"Rv2377c",
"Rv2378c",
"Rv2379c",
"Rv2380c",
"Rv2381c",
"Rv2382c",
"Rv2383c",
"Rv2384",
"Rv2385",
"Rv2386c",
]
DF_X = DF_X[ELIZA_COLUMNS]

In [7]:
CLASSIFIER = classifier_ensemble.ClassifierEnsemble(
        classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=20, size=100)
CLASSIFIER.fit(DF_X, SER_Y)

In [8]:
# GO Terms for the genes
df_go_terms = PROVIDER.df_go_terms.set_index("GENE_ID")
for gene in CLASSIFIER.features:
    if gene in df_go_terms.index:
        print("%s: %s" % (gene, df_go_terms.loc[gene, "GO_Term"]))

Rv2385: triglyceride lipase activity
Rv2386c: tryptophan biosynthetic process---cellular response to ironstarvation---siderophore biosynthetic process from catechol---growth---response-to-host-immune-response---magnesium ion binding---anthranilate synthase activity---chorismate mutase activity---plasma membrane---anthranilate synthase complex
Rv3841: response to hypoxia---iron assimilation by chelation and transport---response to nitrosative stress---ferroxidase activity---iron ion binding---extracellular region---cell wall---cytosol---plasma membrane
Rv2384: cellular response to ironstarvation---siderophore biosynthetic process from catechol---acid-amino acid ligase activity
Rv2382c: growth
Rv2379c: cell wall---cytosol---plasma membrane
Rv2378c: siderophore biosynthetic process from catechol---growth
Rv2383c: pathogenesis---cellular response to ironstarvation---siderophore biosynthetic process from catechol---response-to-host-immune-response---acyl carrier activity---protein binding--

## Helper Functions

In [9]:
def getStage(states):
    """
    Provides the name of the stage for the value of a state.
    
    Parameters
    ----------
    states: list-int
    """
    timepoints = []
    for state in states:
        timepoint = [i for i, v in SER_Y.iteritems() if v == state][0]
        timepoints.append(timepoint)
    return list(PROVIDER.getStages(timepoints))

# TESTS
assert(getStage([0, 1])[0] == "Transition")

In [10]:
def plotExpressionLevels(df_X, features, ser_y=None, is_plot=True, title=""):
    """
    Heat map of expression levels. Shades states.
    
    Parameters
    ----------
    df_X: DataFrame (feature vector)
    features: list-str
    """
    # Internal constants
    ROTATION = 30
    FONTSIZE = 14
    # Shade replications
    fig, ax = plt.subplots(1, figsize=(20, 5))
    columns = list(features)
    columns.sort()
    try:
        new_df_X = df_X[columns]
    except:
        import pdb; pdb.set_trace()
        pass
    sns.heatmap(new_df_X.T, cmap="seismic", ax=ax, vmin=-1, vmax=1)
    # Shade the classes
    if ser_y is not None:
        alphas = [0.0, 0.4]
        alpha_idx = 0
        for idx, val in enumerate(ser_y.values):
            stage = getStage([val])[0]
            if (idx == 0):
                ax.text(idx, 0, "%s" % stage, fontsize=FONTSIZE, rotation=ROTATION)
            elif ser_y.values[idx-1] != val:
                ax.text(idx, 0, "%s" % stage, fontsize=FONTSIZE, rotation=ROTATION)
                alpha_idx = 1 - alpha_idx
            ax.axvspan(idx, idx+1, facecolor='grey', alpha=alphas[alpha_idx])
    # Other plot characteristics
    ax.set_title(title, fontsize=18)
    #
    if is_plot:
        plt.show()
    else:
        plt.close()
    
# Tests
plotExpressionLevels(DF_X, CLASSIFIER.features, SER_Y, is_plot=False)
print("OK!")

OK!


In [11]:
def sortColumns(df):
    if df is None:
        return None
    columns = list(df.columns)
    columns.sort()
    df.columns = columns
    df = df.sort_index()
    return df

In [14]:
def getSampleData(**kwargs):
    sample_data = trinary_data.getSampleData(**kwargs)
    dct = {}
    for sample in trinary_data.SAMPLES:
        dct[sample] = sortColumns(sample_data.__getattribute__(sample))
    return dct
        
SAMPLE_DATA_DCT = getSampleData(is_regulator=False, ref_type=REF_TYPE_BIOREACTOR)
print("OK!")

OK!


In [23]:
def mkDataSubset(genes, selected_stages=None):
    """
    Creates a subset of the data for the genes and stages.
    
    Returns
    -------
    TrinaryData
    """
    data = copy.deepcopy(NEW_DATA)
    data.df_X = pd.DataFrame(data.df_X[genes])
    if selected_stages is not None:
        ser_y = data.ser_y.copy()
        numeric_stages = []
        value_dct = {i: 1 if (s in selected_stages) else 0 for  i, s in enumerate(STAGE_NAMES)}
        new_values = [value_dct[v] for v in data.ser_y]
        data.ser_y = pd.Series(new_values)
        data.ser_y.index = ser_y.index
    return data

# Tests
genes = ["Rv3841", "Rv2382c"]
data = mkDataSubset(genes, selected_stages=["StageII"])
assert(len(data.df_X.columns) == len(genes))
assert(data.ser_y.max() == 1)

> <ipython-input-23-f614e25d911f>(17)mkDataSubset()
-> data.ser_y = pd.Series(new_values)


(Pdb)  n


> <ipython-input-23-f614e25d911f>(18)mkDataSubset()
-> data.ser_y.index = ser_y.index


(Pdb)  ser_y.index


<built-in method index of list object at 0x7f46a7da8c08>


(Pdb)  type(ser_y)


<class 'list'>


(Pdb)  exit


BdbQuit: 

## Data

### BioReactor

In [ ]:
DF_X.head(2)

In [ ]:
SER_Y.head(2)

In [ ]:
# Deeper look at training data
plotExpressionLevels(DATA.df_X, CLASSIFIER.features, ser_y=SER_Y, title="")

In [ ]:
instances = ["T2"]
for instance in instances:
    CLASSIFIER.plotFeatureContributions(DF_X.loc[instance, :],
                    title=instance, true_class=SER_Y.loc[instance],
                    class_names=STAGE_NAMES)

In [ ]:
len(CLASSIFIER.columns)

In [ ]:
def evaluateClassifierOnInstances(clf, df_X=NEW_DATA.df_X, ser_y=None, class_names=None, is_plot=True, nrow=6, ncol=4, suptitle=""):
    _, axes = plt.subplots(nrow, ncol, figsize=(18,12))
    for irow in range(nrow):
        for icol in range(ncol):
            ax = axes[irow, icol]
            instance_num = irow*ncol + icol + 2
            instance = "T%d" % instance_num
            if instance not in df_X.index:
                break
            ser_X = df_X.loc[instance, :]
            if (icol + 1 == ncol) and (irow==0):
                is_legend = True
            else:
                is_legend = False
            if (irow + 1 == nrow):
                is_xlabel = True
                if icol == 0:
                    is_ylabel = True
                else:
                    is_ylabel = False
            else:
                is_ylabel = False
                is_xlabel = False
            clf.plotFeatureContributions(ser_X, ax=ax,
                    title=instance, true_class=ser_y.loc[instance], is_plot=False, is_legend=is_legend,
                                               class_names=class_names,
                                               is_xlabel=is_xlabel, is_ylabel=is_ylabel)
    plt.suptitle(suptitle)
    if not is_plot:
        plt.close()
    
# Tests
evaluateClassifierOnInstances(CLASSIFIER, ser_y=NEW_DATA.ser_y, class_names=STAGE_NAMES, is_plot=True, nrow=2, ncol=2, suptitle="test")

In [ ]:
evaluateClassifierOnInstances(CLASSIFIER, ser_y=NEW_DATA.ser_y, class_names=STAGE_NAMES, nrow=4, ncol=6)

### Lab Samples

In [ ]:
if False:
    df = trinary_data.getSampleData(is_regulator=False).GSE167232
    for _ in range(20):
        sample_data = trinary_data.getSampleData(is_regulator=False)
        if not (df.equals(sample_data.GSE167232)):
            import pdb; pdb.set_trace()
            pass

In [ ]:
def plotSampleData(genes=CLASSIFIER.features, is_plot=True, **kwargs):
    """
    Plots trinary values of data for the specified genes.
    
    Parameters
    ----------
    genes: list-str
    kwargs: optional arguments for getSampleData
    """
    sample_data = trinary_data.getSampleData(**kwargs)
    for name in trinary_data.SAMPLES:
        if name == "sherman":
            continue
        df = sample_data.__getattribute__(name)
        if df is not None:
            plotExpressionLevels(df, genes, title=name, is_plot=is_plot)
            
# Tests
plotSampleData(is_regulator=False, ref_type=REF_TYPE_BIOREACTOR, is_plot=False)
print("OK!")

In [ ]:
# With Bioreactor reference data
plotSampleData(is_regulator=False, ref_type=REF_TYPE_BIOREACTOR)

In [ ]:
# With pooled reference data
plotSampleData(is_regulator=False, ref_type=REF_TYPE_POOLED)

# Classification Validations on Training Samples
Classify T2-T25 and see if result is same as original class. Use 5-fold cross validation, where there is a holdout for each class and the selection is random.


In [ ]:
if False:
    accuracy_dct = {}
    ranks = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    ranks = range(1,12)
    for rank in ranks:
        accuracy_dct[rank] = CLASSIFIER.crossValidate(
            NEW_DATA, num_iter=10, num_holdout=1, filter_high_rank=rank, size=100)
    plt.plot(list(accuracy_dct.keys()), list(accuracy_dct.values()))
    plt.ylim([0, 1.1])
    _ = plt.xlabel("No genes in clasifier")
    _ = plt.ylabel("accuracy")

**TODO**: List the genes used for each coordinate.

In [ ]:
CLASSIFIER.plotSVMCoefficients(title="Classifier Coefficients", class_names=STAGE_NAMES)

The above chart displays the values of the SVM coefficients for each of the genes for the classifier used to identify each state. The vertical lines are error bars ($\pm$ 1 standard deviation) for the value of the coefficient over 100 classifiers constructed using 1 holdout per state.

The coefficients are used to identify the stage that best matches a gene expression vector. Given
an expression vector $\mathbf{x} = (x_1, \cdots, x_N)$, where $x_n \in [-1, 1 ]$ is the expression level of gene $x_n$ and
$c_{m,n}$ is the coefficient for the classifier for stage $m$ and gene $n$:
1. $d_m = \sum_n c_{m,n} x_n$
1. Choose stage $m^{\star}$ such that $m^{\star} = arg max_m \{ d_m \}$

In [ ]:
# Deeper look at training data
plotExpressionLevels(DATA.df_X, CLASSIFIER.features, ser_y=SER_Y, title="")

Several observations are of interest:
1. The coefficients for Transition are mostly negative, which means that this stage is about *under-expression*.
In particular, this stage is characterized by an under-expression of the genes Rv2383c, Rv2382c, Rv2386c, and Rv2385.
1. We see that Stage1a is characterized by large positive values of the coefficients for Rv2385, Rv3841, and Rv2384.
Although this makes sense for Rv2385, it is counter intuitive for
Rv2384 and Rv3841.
The rationale here is more complicated.
Rv2384 is not differentially expressed in Stage 1a; however, it is under-expressed in Transition.
So, a positive coefficient for Rv2384 for Stage 1a allows for differentiation from Transition.
A similar tactic seems to be in play for Rv3841 to distinguish Stage1a from Stage II and Resuscitation.
However, this is complicated by the fact that Transition over-expresses Rv3841.
The classifier compensates for this by having a large coefficient for Rv2384.

1. The other stages are characterized by a mixture of over- and under-expression.

In [ ]:
CLASSIFIER.clfs[0].coef_

# Accuracy of Gene Groups

In [ ]:
def plotAccuracy(genes, title="", selected_stages=None, size=100, is_plot=True, ax=None):
    """
    Plots the accuracy curve for a collection of genes
    
    Parameters
    ----------
    genes: list-str
    title: str
    selected_stages: list-str
        Create a binary classifier for just these tages
    size: int
    is_plot: bol
    """
    if ax is None:
        _, ax = plt.subplots(1)
    accuracy_dct = {}
    data = mkDataSubset(genes, selected_stages=selected_stages)
    ranks = range(1, len(genes) + 1)
    for rank in ranks:
        accuracy_dct[rank] = CLASSIFIER.crossValidate(
            data, num_iter=10, num_holdout=1, filter_high_rank=rank, size=size)
    ax.plot(list(accuracy_dct.keys()), list(accuracy_dct.values()), marker="o")
    ax.set_ylim([0, 1.1])
    _ = ax.set_xlabel("No genes in clasifier")
    _ = ax.set_ylabel("accuracy")
    ax.set_title(title)
    if not is_plot:
        plt.close()
    return accuracy_dct
    
# Tests
genes = GENE_DCT["bacterioferritin 1"]
accuracy_dct = plotAccuracy(GENE_DCT["bacterioferritin 1"], title=", ".join(genes), selected_stages="StageII", size=5, is_plot=False)
assert(np.isclose(list(accuracy_dct.values())[0], 0.5))

In [ ]:
if True:
    gene_dct = {g: [g] for g in GENE_DCT["mycobactin"]}
    gene_dct.update(GENE_DCT)
    for name, genes in gene_dct.items():
        _, ax = plt.subplots(1)
        _ = accuracy_dct = plotAccuracy(genes, title=", ".join(genes), selected_stages=["StageII"], ax=ax, size=100)

In [ ]:
gene_dct = {g: [g] for g in GENE_DCT["mycobactin"]}
gene_dct.update(GENE_DCT)
for name, genes in gene_dct.items():
    _, ax = plt.subplots(1)
    _ = accuracy_dct = plotAccuracy(genes, title=", ".join(genes), ax=ax, size=100)

In [ ]:
# Details for the 10 genes
df_X = DF_X[GENE_DCT["mycobactin"]]
num_genes = range(1, len(df_X.columns) + 1)
num_genes = [10]
for num_gene in num_genes:
    clf = classifier_ensemble.ClassifierEnsemble(
            classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=num_gene, size=100)
    clf.fit(df_X, SER_Y)
    evaluateClassifierOnInstances(clf, df_X=df_X, ser_y=NEW_DATA.ser_y, class_names=STAGE_NAMES, nrow=4, ncol=6, is_plot=True)

## Evaluation for good performing genes

In [ ]:
ser_X = pd.Series([1])
ser_X.index = ["Rv2384"]
df_X = pd.DataFrame(ser_X).T
clf.predict(pd.DataFrame(df_X))

In [ ]:
if False:
    genes = ["Rv2384"]
    data = mkDataSubset(genes, selected_stages=["StageII"])
    clf = classifier_ensemble.ClassifierEnsemble(
            classifier_ensemble.ClassifierDescriptorSVM(), filter_high_rank=20, size=100)
    clf.fit(data.df_X, data.ser_y)
    clf.plotSVMCoefficients(title="Classifier Coefficients", class_names=["Not StageII", "StageII"])

# Classification of Samples

In [ ]:
CLASSIFIER.features

In [ ]:
CLASSIFIER.predict(SAMPLE_DATA_DCT["AM_MDM"])

In [ ]:
stage_names = STAGE_NAMES[np.array(CLASSIFIER.classes)]
stage_names

In [ ]:
def reportSamplePredictions(sample_data):
    for name in trinary_data.SAMPLES:
        df_X = sample_data.__getattribute__(name)
        print("\n%s" % name)
        df_pred = CLASSIFIER.predict(df_X)
        new_columns = [STAGE_NAMES[c] for c in df_pred.columns]
        df_pred.columns = new_columns
        print(df_pred)

In [ ]:
sample_data = trinary_data.getSampleData(is_regulator=False, ref_type=REF_TYPE_POOLED)
reportSamplePredictions(sample_data)

In [ ]:
sample_data.__getattribute__("AW")

# Detailed Analyses

## GSE167232

In [ ]:
plotExpressionLevels(SAMPLE_DATA_DCT["GSE167232"], CLASSIFIER.features, title="GSE167232")

In [ ]:
#CLASSIFIER.plotFeatureContributions(df_X.loc["TB_AM1",:], class_names=STAGE_NAMES)
df_predict = CLASSIFIER.predict(SAMPLE_DATA_DCT["GSE167232"])
indexArr = np.reshape(list(df_predict.index), (4,3))
num_row = 4
num_col =3
_, axes = plt.subplots(num_row, num_col, figsize=(20, 12))
for irow in range(4):
    for icol in range(3):
        if icol == 0:
            is_ylabel = True
        else:
            is_ylabel = False
        if icol == num_col - 1:
            is_legend = True
        else:
            is_legend = False
        if irow == num_row - 1:
            is_xlabel = True
        else:
            is_xlabel = False
        instance = indexArr[irow, icol]
        ser_X = SAMPLE_DATA_DCT["GSE167232"].loc[instance, :]
        CLASSIFIER.plotFeatureContributions(ser_X, class_names=STAGE_NAMES, title=instance, ax=axes[irow,icol], is_plot=False,
                                           is_xlabel=is_xlabel, is_ylabel=is_ylabel, is_legend=is_legend)
plt.show()

In [ ]:
df_X

## Rustad

In [ ]:
# Rustad analysis
df_X = SAMPLE_DATA_DCT["rustad"]
df_pred = CLASSIFIER.predict(df_X)
instances = [i for i in df_pred.index if "H37Rv_hypoxia" in i]
df_pred = df_pred.loc[instances, :]
times = []
replications = []
for instance in instances:
    if "hr" in instance:
        pos = instance.index("hr_")
    else:
        pos = instance.index("ay_")
    last_pos = pos + 2
    times.append(instance[0:last_pos])
    if "10" in instance:
        replications.append(instance[-2:])
    else:
        replications.append(instance[-1])
timeArr = np.array(times)
replicationArr = np.array(replications)
indexArr = list(df_pred.index)
indexArr = np.array(indexArr)
#
state_dct = {}
for time in set(times):
    idxs = [p for p, i in enumerate(indexArr) if time in i]
    replications_sub = replicationArr[idxs]
    times_sub = timeArr[idxs]
    state_dct[time] = []
    for instance in indexArr[idxs]:
        max_val = np.max(df_pred.loc[instance, :].values)
        state = [s for s in df_pred.columns if df_pred.loc[instance, s] == max_val][0]
        state_dct[time].append(STAGE_NAMES[state])

In [ ]:
time_keys = np.array(["H37Rv_hypoxia_4hr", "H37Rv_hypoxia_8hr", "H37Rv_hypoxia_12hr",
                      "H37Rv_hypoxia_1day", "H37Rv_hypoxia_4day", "H37Rv_hypoxia_7day"])
truncated_time_keys = np.array(["4hr", "8hr", "12hr",
                      "1day", "4day", "7day"])
time_idxs = np.array([list(state_dct.keys()).index(k) for k in time_keys])
repl_dct = {}
for repl in [0, 1, 2]:
    repl_dct[repl] = [state_dct[t][repl] for t in time_keys[time_idxs]]

In [ ]:
_, ax = plt.subplots(1)
repl_dct[-1] = ["Transition", "Stage1a", "Stage1b", "StageII", "Resuscitation"]
repl_dct[-1] = repl_dct[0]
repl_dct[-1][0] = "Transition"
repl_dct[-1][1] = "Stage1a"
repl_dct[-1][2] = "Stage1b"
repl_dct[-1][3] = "StageII"
repl_dct[-1][4] = "Resuscitation"
colors = ["red", "blue", "green"]
for repl in range(-1, 3):
    if repl == -1:
        color = "black"
    else:
        color = colors[repl]
    ax.plot(truncated_time_keys, repl_dct[repl], color=color)
plt.legend([0, 1, 2], loc="upper left")
plt.xlabel("time")
plt.ylabel("Stage")

## AM, MDM

In [ ]:
SAMPLE_DATA_DCT["AM_MDM"]

## Galagan

**TODO**: Plot progression of replications over time